# **Data initialisation**

In [1]:
# Installer les dépendances
!pip install python-dotenv
!pip install supabase
!pip install pandas-profiling
!pip install category_encoders
!pip install mlflow
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.3 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143876 sha256=00bfdb62e58c245bd4ef5cd2fb728ec6c923d95d4f70746419fb3de0e0cd444c
  Stored in directory: /root/.

In [2]:
# Importer les bibliothèques nécessaires
import requests
import csv
import os
import mlflow
import time
import pickle
import glob
import warnings
import numpy as np
import pandas as pd
import category_encoders as ce
from google.colab import drive
from supabase import create_client, Client
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from pandas_profiling import ProfileReport
from pyngrok import ngrok

<ipython-input-2-a4e6a45d6525>:20: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [3]:
# Configurer l'environnement
warnings.filterwarnings('ignore')
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Projet MLOps/')
load_dotenv('config.env')

Mounted at /content/drive


True

# **Data extraction**

In [4]:
# data_extraction.py
def fetch_table_data_columns(supabase, table_name):
    print(f"Récupération des données et des colonnes de la table {table_name}...")
    response = supabase.table(table_name).select("*").execute()
    data = response.data
    if not data:
        print(f"Aucune donnée trouvée pour la table {table_name}.")
        return None, None
    columns = data[0].keys()
    print(f"Données récupérées avec succès pour la table {table_name}.")
    return data, columns

def write_data_to_csv(data, columns, output_file):
    print(f"Écriture des données au format CSV dans le fichier {output_file}...")
    with open(output_file, mode='w', encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        if data:
            for row in data:
                writer.writerow(row)
    print(f"Exportation réussie vers {output_file}.")

def export_table_to_csv(supabase, table_name, output_file):
    print("Début de l'exportation de la table au format CSV...")
    data, columns = fetch_table_data_columns(supabase, table_name)
    if data:
        write_data_to_csv(data, columns, output_file)
    print("Exportation terminée.")

def get_csv_links(url):
    print("Récupération des liens CSV depuis l'URL...")
    try:
        html_content = requests.get(url).text
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de l'URL : {e}")
        return []
    soup = BeautifulSoup(html_content, 'html.parser')
    return [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.csv')]

def get_downloaded_files_list(filepath):
    print("Récupération de la liste des fichiers téléchargés...")
    try:
        with open(filepath, "r") as f:
            return f.read().splitlines()
    except FileNotFoundError:
        return []

def save_downloaded_files_list(downloaded_files, filepath):
    print("Sauvegarde de la liste des fichiers téléchargés...")
    with open(filepath, "w") as f:
        f.write("\n".join(downloaded_files))

def download_new_files(csv_links, file_patterns, downloaded_files):
    print("Téléchargement des nouveaux fichiers...")
    new_files = []
    for link in csv_links:
        filename = os.path.basename(link)
        if any(pattern in filename for pattern in file_patterns):
            if filename not in downloaded_files:
                print(f"Téléchargement du fichier : {filename}")
                try:
                    content = requests.get(link).content
                except requests.exceptions.RequestException as e:
                    print(f"Erreur lors du téléchargement du fichier {filename} : {e}")
                    continue
                with open(filename, "wb") as f:
                    f.write(content)
                new_files.append(filename)
        else:
            print(f"Le fichier {filename} ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.")
    return new_files

In [5]:
# Tâche pour extraire les données "live" à partir de Supabase
def data_extraction_live_func():
    url: str = os.getenv("SUPABASE_URL")
    key: str = os.getenv("SUPABASE_KEY")
    supabase: Client = create_client(url, key)

    table_name = "data_live"
    output_file = "data_live.csv"
    export_table_to_csv(supabase, table_name, output_file)

# Tâche pour extraire les nouvelles données historiques à partir d'un site Web
def data_extraction_historical_new_func():
    url = "https://www.data.gouv.fr/en/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/"
    csv_links = get_csv_links(url)

    downloaded_files_list_filepath = "downloaded_files.txt"
    downloaded_files = get_downloaded_files_list(downloaded_files_list_filepath)

    file_patterns = ['usagers-', 'lieux-', 'cteristiques-', 'vehicules-']
    new_files = download_new_files(csv_links, file_patterns, downloaded_files)

    downloaded_files += new_files
    save_downloaded_files_list(downloaded_files, downloaded_files_list_filepath)

# Tâche pour extraire les anciennes données historiques à partir de Supabase
def data_extraction_historical_old_func():
    url: str = os.getenv("SUPABASE_URL")
    key: str = os.getenv("SUPABASE_KEY")
    supabase: Client = create_client(url, key)

    table_name = "data_historical_old"
    output_file = "data_historical_old.csv"
    export_table_to_csv(supabase, table_name, output_file)

In [6]:
data_extraction_live_func()
data_extraction_historical_new_func()
data_extraction_historical_old_func()

Début de l'exportation de la table au format CSV...
Récupération des données et des colonnes de la table data_live...
Aucune donnée trouvée pour la table data_live.
Exportation terminée.
Récupération des liens CSV depuis l'URL...
Récupération de la liste des fichiers téléchargés...
Téléchargement des nouveaux fichiers...
Le fichier lieux.csv ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.
Le fichier vehicules.csv ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.
Le fichier caracteristiques.csv ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.
Le fichier usagers.csv ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.
Le fichier accidents-corp-2019.csv ne correspond à aucun des motifs de fichier attendus et ne sera pas téléchargé.
Sauvegarde de la liste des fichiers téléchargés...
Début de l'exportation de la table au format CSV...
Récupération des données et des colonn

# **Data merging**

In [7]:
# data_merging.py
def find_most_recent_csv_file(pattern: str):
    print(f"Recherche du fichier CSV correspondant au motif '{pattern}'...")
    matching_files = glob.glob(pattern)
    if not matching_files:
        return None
    most_recent_file = max(matching_files, key=os.path.getmtime)
    return most_recent_file

def find_csv_delimiter(file_path: str, sample_size: int = 1024, encoding: str = 'latin1'):
    print(f"Détection du délimiteur pour le fichier '{file_path}'...")
    try:
        with open(file_path, 'r', encoding=encoding) as csvfile:
            sample = csvfile.read(sample_size)
            sniffer = csv.Sniffer()
            delimiter = sniffer.sniff(sample).delimiter
    except Exception as e:
        print(f"Erreur lors de la détection du délimiteur : {e}")
        return None
    print(f"Délimiteur détecté: '{delimiter}'")
    return delimiter

def read_csv_with_delimiter(file_path: str, delimiter: str, encoding: str = 'latin1', low_memory: bool = False):
    print(f"Lecture du fichier CSV '{file_path}' avec le délimiteur '{delimiter}'...")
    try:
        return pd.read_csv(file_path, sep=delimiter, encoding=encoding, low_memory=low_memory)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier CSV : {e}")
        return None

def merge_dataframes(df_list, merge_keys, merge_how='left'):
    if not df_list:
        print("Aucun DataFrame à fusionner.")
        return None
    if len(df_list) == 1:
        print("Un seul DataFrame présent, pas besoin de fusion.")
        return df_list[0]

    print("Fusion des DataFrames...")
    merged_data = df_list[0]
    for i in range(1, len(df_list)):
        print(f"Fusion des DataFrames {i} et {i+1} sur la clé {merge_keys[i-1]}...")
        merged_data = pd.merge(merged_data, df_list[i], how=merge_how, on=merge_keys[i-1])
    print("Fusion terminée.")
    return merged_data

In [8]:
# Tâche pour fusionner les nouvelles données historiques, les anciènnes données historiques et les données "live"
def data_merging_func():
    usagers_file = find_most_recent_csv_file("*usagers*.csv")
    lieux_file = find_most_recent_csv_file("*lieux*.csv")
    caracteristiques_file = find_most_recent_csv_file("*cteristiques*.csv")
    vehicules_file = find_most_recent_csv_file("*vehicules*.csv")

    if usagers_file and lieux_file and caracteristiques_file and vehicules_file:
        delimiter_usagers = find_csv_delimiter(usagers_file)
        delimiter_lieux = find_csv_delimiter(lieux_file)
        delimiter_caracteristiques = find_csv_delimiter(caracteristiques_file)
        delimiter_vehicules = find_csv_delimiter(vehicules_file)

        data_usagers = read_csv_with_delimiter(usagers_file, delimiter_usagers)
        data_lieux = read_csv_with_delimiter(lieux_file, delimiter_lieux)
        data_caracteristiques = read_csv_with_delimiter(caracteristiques_file, delimiter_caracteristiques)
        data_vehicules = read_csv_with_delimiter(vehicules_file, delimiter_vehicules)

        dataframes = [data_usagers, data_vehicules, data_caracteristiques, data_lieux]
        merge_keys = [["Num_Acc", "num_veh"], ["Num_Acc"], ["Num_Acc"]]
        data_historical_new = merge_dataframes(dataframes, merge_keys)
        data_historical_new.to_csv("data_historical_new.csv", index=False, encoding='utf-8')
        data_historical_old = pd.read_csv("data_historical_old.csv", sep=',', encoding='latin1', low_memory=False)
        data_historical = pd.concat([data_historical_new, data_historical_old], axis=0, ignore_index=True)
    else:
        data_historical = pd.read_csv("data_historical_old.csv", sep=',', encoding='latin1', low_memory=False)

    if os.path.isfile("data_live.csv"):
        data_live = pd.read_csv("data_live.csv", sep=',', encoding='latin1', low_memory=False)
        data_accidents = pd.concat([data_historical, data_live], axis=0, ignore_index=True)
    else:
        data_accidents = data_historical

    data_accidents.to_csv("data_accidents.csv", index=False, encoding='utf-8')

In [9]:
data_merging_func()

Recherche du fichier CSV correspondant au motif '*usagers*.csv'...
Recherche du fichier CSV correspondant au motif '*lieux*.csv'...
Recherche du fichier CSV correspondant au motif '*cteristiques*.csv'...
Recherche du fichier CSV correspondant au motif '*vehicules*.csv'...


# **Data profiling**

In [10]:
# data_profiling.py
def generate_data_profile(df: pd.DataFrame, output_file: str, title: str):
    print("Création du profile...")
    profile = ProfileReport(
        df,
        title=title,
        minimal=True,
        correlations=None,
        html={'style': {'full_width': True}},
    )
    print("Génération du rapport...")
    profile.to_file(output_file=output_file)
    print(f"Le rapport a été généré avec succès et enregistré dans le fichier {output_file}.")

In [11]:
## Tâche pour générer un rapport d'analyse exploratoire des données
def data_profiling_func():
    data_file = "data_accidents.csv"
    df = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    output_file = "data-profile.html"
    title = "Analyse exploratoire des accidents routiers en France"
    generate_data_profile(df, output_file, title)

In [12]:
data_profiling_func()

Création du profile...
Génération du rapport...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Le rapport a été généré avec succès et enregistré dans le fichier data-profile.html.


# **Data processing (all features)**

In [13]:
# data_processing.py
def preprocess_data1(df: pd.DataFrame, target):
    print("Suppression des lignes dont la valeur de la cible n'est pas 1, 2, 3 ou 4...")
    df = df[df[target].isin([1, 2, 3, 4])]
    
    print("Remplacement des valeurs manquantes par le mode de chaque groupe de la cible...")
    df = df.groupby(target).apply(lambda x: x.fillna(x.mode().iloc[0])).reset_index(drop=True)
    
    print("Conversion de la cible en 0 si elle est égale à 1 ou 4, sinon en 1...")
    df[target] = df[target].apply(lambda x: 0 if x in [1, 4] else 1)
    
    colonnes_a_supprimer = [colonne for colonne in df.columns if 'id_vehicule' in colonne]
    if colonnes_a_supprimer:
      df = df.drop(colonnes_a_supprimer, axis=1).drop_duplicates()
    
    return df

def encode_categorical_data1(data, target):
    print("Encodage des colonnes catégorielles avec TargetEncoder...")
    categorical_columns = data.select_dtypes(include=['object'])
    numerical_columns = data.select_dtypes(exclude=['object'])
    encoder = ce.TargetEncoder()
    encoded_data = encoder.fit_transform(categorical_columns, data[target])

    return pd.concat([numerical_columns, encoded_data], axis=1)

In [14]:
## Tâche pour prétraiter et encoder les données
def data_processing1_func():
    data_file = "data_accidents.csv"
    df = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    target = "grav"
    df_preprocessed = preprocess_data1(df, target)

    df_encoded = encode_categorical_data1(df_preprocessed, target)

    output_file = "data_accidents_encoded.csv"
    df_encoded.to_csv(output_file, index=False, encoding='utf-8')

In [15]:
data_processing1_func()

Suppression des lignes dont la valeur de la cible n'est pas 1, 2, 3 ou 4...
Remplacement des valeurs manquantes par le mode de chaque groupe de la cible...
Conversion de la cible en 0 si elle est égale à 1 ou 4, sinon en 1...
Encodage des colonnes catégorielles avec TargetEncoder...


# **Data featuring**

In [16]:
def train_model(experiment_name, df_encoded, target):
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print("L'expérience a été créée.")
    else:
        experiment_id = current_experiment.experiment_id
        print("L'expérience existe déjà.")

    with mlflow.start_run(experiment_id = experiment_id):
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Les données ont été préparées.")

        model = XGBClassifier()
        model.fit(X_train, y_train)
        print("Le modèle a été entraîné avec les features sélectionnées.")

        importances = model.feature_importances_
        feature_importances = sorted(zip(X.columns, importances), key=lambda x: x[1], reverse=True)
        selected_features = [feature for feature, importance in feature_importances[:15]]
        print(f"Les 15 features les plus importantes sont : {selected_features}")

        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_param("top15_feature", selected_features)
        print("Résultats de l'entraînement sauvegardés")
        
    print("Entraînement terminé")

In [17]:
## Tâche pour entraîner et évaluer le modèle de prédiction
def data_featuring_func():
    data_file = "data_accidents_encoded.csv"
    df_encoded = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    experiment_name = "Accidents_Model"
    target = "grav"
    train_model(experiment_name, df_encoded, target)

In [18]:
data_featuring_func()

L'expérience a été créée.
Les données ont été préparées.
Le modèle a été entraîné avec les features sélectionnées.
Les 15 features les plus importantes sont : ['lartpc', 'occutc', 'lat', 'com', 'adr', 'mois', 'pr1', 'pr', 'voie', 'dep', 'hrmn', 'col', 'agg', 'choc', 'vma']
Résultats de l'entraînement sauvegardés
Entraînement terminé


# **Data processing (top 15 features)**

In [19]:
# data_processing.py
def preprocess_data2(df: pd.DataFrame, target, selected_features):
    print("Suppression des lignes dont la valeur de la cible n'est pas 1, 2, 3 ou 4...")
    df = df[df[target].isin([1, 2, 3, 4])]
    
    print("Remplacement des valeurs manquantes par le mode de chaque groupe de la cible...")
    df = df.groupby(target).apply(lambda x: x.fillna(x.mode().iloc[0])).reset_index(drop=True)
    
    print("Conversion de la cible en 0 si elle est égale à 1 ou 4, sinon en 1...")
    df[target] = df[target].apply(lambda x: 0 if x in [1, 4] else 1)
    
    df = df[selected_features].drop_duplicates()
    
    return df

def encode_categorical_data2(data, target):
    print("Encodage des colonnes catégorielles avec TargetEncoder...")
    categorical_columns = data.select_dtypes(include=['object'])
    numerical_columns = data.select_dtypes(exclude=['object'])
    encoder = ce.TargetEncoder()
    encoded_data = encoder.fit_transform(categorical_columns, data[target])

    timestamp = time.strftime('%Y%m%d_%H%M%S')
    filename = f"targetencoder_v2.0_{timestamp}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(encoder, file)
    print("Encodeur entraîné sauvegardé")

    return pd.concat([numerical_columns, encoded_data], axis=1)

In [20]:
## Tâche pour prétraiter et encoder les données
def data_processing2_func():
    data_file = "data_accidents.csv"
    df = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    target = "grav"
    selected_features = ['long', 'lat', 'secu1', 'etatp', 'agg', 'place', 'obsm', 'catv', 'catu', 'sexe', 'voie', 'col', 'catr', 'obs', 'motor', 'grav']
    df_preprocessed = preprocess_data2(df, target, selected_features)

    df_encoded = encode_categorical_data2(df_preprocessed, target)

    output_file = "data_accidents_encoded.csv"
    df_encoded.to_csv(output_file, index=False, encoding='utf-8')

In [21]:
data_processing2_func()

Suppression des lignes dont la valeur de la cible n'est pas 1, 2, 3 ou 4...
Remplacement des valeurs manquantes par le mode de chaque groupe de la cible...
Conversion de la cible en 0 si elle est égale à 1 ou 4, sinon en 1...
Encodage des colonnes catégorielles avec TargetEncoder...
Encodeur entraîné sauvegardé


# **Data modeling**

In [22]:
# data_modeling.py
def train_model(experiment_name, df_encoded, target):
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print("L'expérience a été créée.")
    else:
        experiment_id = current_experiment.experiment_id
        print("L'expérience existe déjà.")

    with mlflow.start_run(experiment_id = experiment_id):
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Les données ont été préparées.")

        model = XGBClassifier()
        model.fit(X_train, y_train)
        print("Le modèle a été entraîné avec les features sélectionnées.")

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average=None)
        recall = recall_score(y_test, y_pred, average=None)
        f1_scores = f1_score(y_test, y_pred, average=None)
        print("Le modèle a été évalué avec les métriques de performance.")

        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_metric("accuracy", accuracy)
        for label, p in enumerate(precision):
            mlflow.log_metric(f"precision_class_{label}", p)
        for label, r in enumerate(recall):
            mlflow.log_metric(f"recall_class_{label}", r)
        for label, f1 in enumerate(f1_scores):
            mlflow.log_metric(f"f1_score_class_{label}", f1)
        print("Résultats de l'entraînement sauvegardés")

        timestamp = time.strftime('%Y%m%d_%H%M%S')
        filename = f"xgboost_v2.0_{timestamp}.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(model, file)
        print("Modèle entraîné sauvegardé")

        mlflow.log_artifact(filename)
        print("Modèle sauvegardé dans MLflow")
        
    print("Entraînement terminé")

In [23]:
## Tâche pour entraîner et évaluer le modèle de prédiction
def data_modeling_func():
    data_file = "data_accidents_encoded.csv"
    df_encoded = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    experiment_name = "Accidents_Model"
    target = "grav"
    train_model(experiment_name, df_encoded, target)

In [24]:
data_modeling_func()

L'expérience existe déjà.
Les données ont été préparées.
Le modèle a été entraîné avec les features sélectionnées.
Le modèle a été évalué avec les métriques de performance.
Résultats de l'entraînement sauvegardés
Modèle entraîné sauvegardé
Modèle sauvegardé dans MLflow
Entraînement terminé


# **Data tuning**

In [ ]:
# data_tuning.py
def xgboost_fine_tuning(experiment_name, df_encoded, target, n_iter=100, cv=3):
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print("L'expérience a été créée.")
    else:
        experiment_id = current_experiment.experiment_id
        print("L'expérience existe déjà.")

    with mlflow.start_run(experiment_id = experiment_id):
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Les données ont été préparées.")

        model = XGBClassifier(objective='binary:logistic', random_state=42)

        param_grid = {
            "learning_rate": np.logspace(-3, 0, num=200),
            "max_depth": np.arange(1, 10),
            "n_estimators": np.arange(50, 300, step=10),
            "gamma": np.logspace(-8, 0, num=200),
            "min_child_weight": np.arange(1, 10),
            "subsample": np.linspace(0.5, 1, num=100),
            "colsample_bytree": np.linspace(0.5, 1, num=100),
            "reg_alpha": np.logspace(-8, 0, num=200),
            "reg_lambda": np.logspace(-8, 0, num=200),
        }

        random_search = RandomizedSearchCV(
            model, 
            param_distributions=param_grid, 
            n_iter=n_iter, 
            scoring="neg_mean_squared_error", 
            cv=cv, 
            n_jobs=-1, 
            random_state=42
        )

        random_search.fit(X_train, y_train)

        best_hyperparameters = random_search.best_params_
        print(f"Best hyperparameters: {best_hyperparameters}")

        best_model = random_search.best_estimator_
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average=None)
        recall = recall_score(y_test, y_pred, average=None)
        f1_scores = f1_score(y_test, y_pred, average=None)
        print("Le modèle a été évalué avec les métriques de performance.")

        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_param("method_optimazer", "RandomizedSearchCV")
        mlflow.log_params(best_hyperparameters)
        mlflow.log_metric("accuracy", accuracy)
        for label, p in enumerate(precision):
            mlflow.log_metric(f"precision_class_{label}", p)
        for label, r in enumerate(recall):
            mlflow.log_metric(f"recall_class_{label}", r)
        for label, f1 in enumerate(f1_scores):
            mlflow.log_metric(f"f1_score_class_{label}", f1)
        print("Résultats de l'entraînement sauvegardés")

        timestamp = time.strftime('%Y%m%d_%H%M%S')
        filename = f"xgboost_v3.0_{timestamp}.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(best_model, file)
        print("Modèle entraîné sauvegardé")

        mlflow.log_artifact(filename)
        print("Modèle sauvegardé dans MLflow")

    print("Entraînement terminé")

In [ ]:
## Tâche pour améliorer les performances du modèle
def data_tuning_func():
    data_file = "data_accidents_encoded.csv"
    df_encoded = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    experiment_name = "Accidents_Model"
    target = "grav"
    xgboost_fine_tuning(experiment_name, df_encoded, target, n_iter=100, cv=2)

In [ ]:
data_tuning_func()

# **Data management**

In [25]:
import os
import time
import glob
import csv
import tarfile
import pandas as pd
from github import Github

def create_destination_folder(source_folder, file_path):
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    destination_folder = os.path.join(file_path, timestamp)
    os.makedirs(destination_folder)
    print(f"Dossier de destination créé : {destination_folder}")
    return destination_folder

def get_artifacts(source_folder):
    file_extensions = ('*.csv', '*.html', '*.pkl')
    artifacts_files = []
    for extension in file_extensions:
        artifacts_files.extend(glob.glob(os.path.join(source_folder, extension)))
    print(f"Nombre de fichiers pris en charge à archiver : {len(artifacts_files)}")
    return artifacts_files

def archive_csv_file(source_file, destination_folder, file_name):
    destination_file = os.path.join(destination_folder, file_name)
    os.rename(source_file, destination_file)
    print(f"Fichier archivé : {source_file} -> {destination_file}")

def read_csv_chunks(csv_file_path, chunk_size=1000):
    with open(csv_file_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.DictReader(csv_file)
        chunk = []
        for row in reader:
            chunk.append(row)
            if len(chunk) >= chunk_size:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

def process_csv_data(csv_file_path, chunk_size=1000):
    print(f"Lecture et traitement du fichier CSV: {csv_file_path}")
    return pd.read_csv(csv_file_path, chunksize=chunk_size, na_filter=False)

def insert_data_to_supabase(supabase, table_name, data_iterator):
    print(f"Insertion des données dans la table '{table_name}'")
    row_count = 0
    for chunk in data_iterator:
        chunk_data = chunk.to_dict(orient='records')
        response = supabase.table(table_name).insert(chunk_data).execute()
        if response:
            print(f"{len(chunk_data)} lignes insérées avec succès")
            row_count += len(chunk_data)
        else:
            print(f"Échec de l'insertion des données : {response.error}")
    print(f"{row_count} lignes insérées au total dans la base de données")

def upload_to_github(token, repo_name, commit_message, branch, files_with_paths):
    g = Github(token)
    repo = g.get_user().get_repo(repo_name)

    for local_file_path, github_file_path in files_with_paths:
        with open(local_file_path, "rb") as file:
            content = file.read()

        repo.create_file(
            path=github_file_path,
            message=commit_message,
            content=content,
            branch=branch
        )

def compress(input_path, output_file):
    destination_directory = os.getcwd()
    output_file_path = os.path.join(destination_directory, output_file)
    with tarfile.open(output_file_path, "w:gz") as tar:
        if os.path.isfile(input_path):
            arcname = os.path.basename(input_path)
        else:
            arcname = ""
        tar.add(input_path, arcname=arcname)

In [26]:
## Tâche pour gérer les données
def data_management_func(ti):
    if os.path.isfile("data_historical_new.csv"):
        csv_file_path = "data_historical_new.csv"
        data_iterator = process_csv_data(csv_file_path)

        url: str = os.getenv("SUPABASE_URL")
        key: str = os.getenv("SUPABASE_KEY")
        supabase: Client = create_client(url, key)

        table_name = "data_historical_old"
        insert_data_to_supabase(supabase, table_name, data_iterator)

    experiment_name = "Accidents_Model"
    alert = data_check.check_metrics_and_alert(experiment_name, ti)
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    if not alert:
        token = os.getenv("GITHUB_TOKEN")
        repo_name = "MlOps_Project"
        commit_message = "Ajout des dernières données d'entrainement"
        branch = "main"
        files_with_paths = [
            ("data_usagers.csv.tar.gz", f"data/raw/{timestamp}/data_usagers.csv.tar.gz"),
            ("data_lieux.csv.tar.gz", f"data/raw/{timestamp}/data_lieux.csv.tar.gz"),
            ("data_caracteristiques.csv.tar.gz", f"data/raw/{timestamp}/data_caracteristiques.csv.tar.gz"),
            ("data_vehicules.csv.tar.gz", f"data/raw/{timestamp}/data_vehicules.csv.tar.gz"),
            ("data_accidents.csv.tar.gz", f"data/interim/{timestamp}/data_accidents.csv.tar.gz"),
            ("data_accidents_encoded.csv.tar.gz", f"data/processed/{timestamp}/data_accidents_encoded.csv.tar.gz"),
        ]
        upload_to_github(token, repo_name, commit_message, branch, files_with_paths)

        token = os.getenv("GITHUB_TOKEN")
        repo_name = "MlOps_Project"
        commit_message = "Ajout du dernier model entrainé"
        branch = "main"
        files_with_paths = [
            ("targetencoder.pkl", f"data/processed/{timestamp}/targetencoder.pkl"),
            ("xgboost.pkl", f"data/processed/{timestamp}/xgboost.pkl"),
        ]
        upload_to_github(token, repo_name, commit_message, branch, files_with_paths)

    compress("mlruns", f"mlrun_{timestamp}.tar.gz")
    token = os.getenv("GITHUB_TOKEN")
    repo_name = "MlOps_Project"
    commit_message = "Ajout des données de l'expérience mlflow"
    branch = "dev"
    files_with_paths = [
        (f"mlrun_{timestamp}.tar.gz", f"experiments/mlrun_{timestamp}.tar.gz"),
    ]
    upload_to_github(token, repo_name, commit_message, branch, files_with_paths)

In [27]:
data_management_func()

Dossier de destination créé : /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124
Nombre de fichiers pris en charge à archiver : 6
Fichier archivé : /content/drive/MyDrive/Projet MLOps/data_historical_old.csv -> /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124/data_historical_old.csv
Fichier archivé : /content/drive/MyDrive/Projet MLOps/data_accidents.csv -> /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124/data_accidents.csv
Fichier archivé : /content/drive/MyDrive/Projet MLOps/data_accidents_encoded.csv -> /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124/data_accidents_encoded.csv
Fichier archivé : /content/drive/MyDrive/Projet MLOps/data-profile.html -> /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124/data-profile.html
Fichier archivé : /content/drive/MyDrive/Projet MLOps/targetencoder_v2.0_20230508_220123.pkl -> /content/drive/MyDrive/Projet MLOps/data_management/20230508_220124/targetencoder

# **Mlflow**

In [31]:
# Configuration et ouverture d'un tunnel ngrok
ngrok.kill()

NGROK_AUTH_TOKEN = "2P4cOaZTTmH9wl5kjQL3Qnn9NIl_6uFwdWZzo8EF6jdLCLef6"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Interface utilisateur MLflow Tracking :", ngrok_tunnel.public_url)

Interface utilisateur MLflow Tracking : https://9320-34-125-254-129.ngrok-free.app


In [32]:
# Lancement de l'interface utilisateur de mlflow
!mlflow ui

[2023-05-08 21:24:03 +0000] [8423] [INFO] Starting gunicorn 20.1.0
[2023-05-08 21:24:03 +0000] [8423] [INFO] Listening at: http://127.0.0.1:5000 (8423)
[2023-05-08 21:24:03 +0000] [8423] [INFO] Using worker: sync
[2023-05-08 21:24:03 +0000] [8428] [INFO] Booting worker with pid: 8428
[2023-05-08 21:24:03 +0000] [8429] [INFO] Booting worker with pid: 8429
[2023-05-08 21:24:03 +0000] [8430] [INFO] Booting worker with pid: 8430
[2023-05-08 21:24:03 +0000] [8431] [INFO] Booting worker with pid: 8431
[2023-05-08 21:26:41 +0000] [8423] [INFO] Handling signal: int

Aborted!
[2023-05-08 21:26:41 +0000] [8428] [INFO] Worker exiting (pid: 8428)
[2023-05-08 21:26:41 +0000] [8430] [INFO] Worker exiting (pid: 8430)
[2023-05-08 21:26:41 +0000] [8431] [INFO] Worker exiting (pid: 8431)
[2023-05-08 21:26:41 +0000] [8429] [INFO] Worker exiting (pid: 8429)
[2023-05-08 21:26:42 +0000] [8423] [INFO] Shutting down: Master
